In [58]:
from pprint import *
import json
import pandas as pd

In [59]:
os.environ["AZURE_CLIENT_ID"] = "client--id"
os.environ["AZURE_CLIENT_SECRET"] = "client--secret"
os.environ["AZURE_TENANT_ID"] = "tenant--id"
os.environ["PURVIEW_NAME"] = "purview--accnt--name"

##### Helpers

In [60]:
import json
def getJSON(raw_output):
    output = ''.join(raw_output)
    json_obj = json.loads(output)
    return json_obj

### Get Assets by Collections

In [61]:
# Get List of Collections
collections = !pv account getCollections
collections = getJSON(collections)

In [62]:
pprint(collections)

{'count': 2,
 'value': [{'collectionProvisioningState': 'Succeeded',
            'description': 'The root collection.',
            'friendlyName': 'aia-purview-new',
            'name': 'aia-purview-new',
            'systemData': {'createdAt': '2021-10-03T15:47:19.0474252Z',
                           'createdBy': 'b99a3530-636d-4621-8662-bc5c8022b125',
                           'createdByType': 'User',
                           'lastModifiedAt': '2021-10-03T15:47:19.0474252Z',
                           'lastModifiedBy': 'b99a3530-636d-4621-8662-bc5c8022b125',
                           'lastModifiedByType': 'User'}},
           {'collectionProvisioningState': 'Succeeded',
            'friendlyName': 'ABCD',
            'name': 'ctukcr',
            'parentCollection': {'referenceName': 'aia-purview-new',
                                 'type': 'CollectionReference'},
            'systemData': {'createdAt': '2022-01-25T23:01:11.9637225Z',
                           'createdBy': '

In [63]:
# Retrieve Collection Name (technical) via Friendly Name
appCode = 'ABCD'
collectionName = None
for collection in collections['value']:
    if collection['friendlyName'] == appCode:
        collectionName = collection['name']
print(f"Friendly Name: {appCode}; Technical Name: {collectionName}")

Friendly Name: ABCD; Technical Name: ctukcr


In [69]:
json_folder = r"C:\Users\mdrrahman\Documents\GitHub\purview-collection-aggregator\purviewcli\json"

my_json = """{
    "and": [
        {
            "collectionId": "%s"
        }
    ]
}
""" % ("aia-purview-new")

print(my_json)

text_file = open(r"{}\filter_collection.json".format(json_folder), "w")
n = text_file.write(my_json)
text_file.close()

{
    "and": [
        {
            "collectionId": "aia-purview-new"
        }
    ]
}



In [93]:
# Search for assets using a filter file that specifies the collection name
filter_file = r"{}\filter_collection.json".format(json_folder)
print(filter_file)
response = !pv search query --filter-file "C:\Users\mdrrahman\Documents\GitHub\purview-collection-aggregator\purviewcli\json\filter_collection.json"

getJSON(response)

C:\Users\mdrrahman\Documents\GitHub\purview-collection-aggregator\purviewcli\json\filter_collection.json


{'@search.count': 19,
 '@search.facets': None,
 'value': [{'@search.highlights': None,
   '@search.score': 36.186295,
   'assetType': ['Azure SQL Database'],
   'classification': ['MICROSOFT.PERSONAL.NAME'],
   'collectionId': 'aia-purview-new',
   'description': None,
   'entityType': 'azure_sql_table',
   'id': 'b9db7f3d-3e24-43a3-86fc-c4f6f6f60000',
   'label': [],
   'name': 'Employees',
   'owner': None,
   'qualifiedName': 'mssql://aemigration.database.windows.net/contosoHR_AE/dbo/Employees',
   'term': [{'glossaryName': 'Glossary',
     'guid': '42fddc35-12e6-49cc-9ae4-b6ade692742b',
     'name': 'Contoso_IC_Internal'},
    {'glossaryName': 'Glossary',
     'guid': '2332b392-e605-4193-9c29-1871f9c4ec74',
     'name': 'Contoso_IC_Public'},
    {'glossaryName': 'Glossary',
     'guid': '5f881ce2-3502-4eec-b446-9006a74cf5d5',
     'name': 'Contoso_IC_Confidential'},
    {'glossaryName': 'Glossary',
     'guid': '0a3751c1-6502-4ee7-a78e-55f5e0ba05b4',
     'name': 'Contoso_IC_Sensit

In [95]:
# Flatten data
df_nested_list = pd.json_normalize(getJSON(response), record_path =['value'])
df_filtered_list = df_nested_list[df_nested_list['collectionId']=="aia-purview-new"]

display(df_filtered_list[['assetType', 'classification', 'collectionId', 'entityType', 'qualifiedName', 'term']])

,assetType,classification,collectionId,entityType,qualifiedName,term
0,[Azure SQL Database],[MICROSOFT.PERSONAL.NAME],aia-purview-new,azure_sql_table,mssql://aemigration.database.windows.net/conto...,"[{'glossaryName': 'Glossary', 'guid': '42fddc3..."
1,[SQL Server],[MICROSOFT.PERSONAL.EMAIL],aia-purview-new,mssql_table,mssql://desktop-1caqu08:11433/MSSQLSERVER/Adve...,[]
2,[SQL Server],[MICROSOFT.PERSONAL.NAME],aia-purview-new,mssql_table,mssql://desktop-1caqu08:11433/MSSQLSERVER/Adve...,[]
3,[Azure Blob Storage],[MICROSOFT.FINANCIAL.EU_DEBIT_CARD],aia-purview-new,azure_blob_path,https://myblobyespe.blob.core.windows.net/scan...,[]
9,[SQL Server],[],aia-purview-new,mssql_instance,mssql://desktop-1caqu08:11433/MSSQLSERVER,[]
10,[SQL Server],[],aia-purview-new,mssql_db,mssql://desktop-1caqu08:11433/MSSQLSERVER/Adve...,[]
11,[SQL Server],[],aia-purview-new,mssql_schema,mssql://desktop-1caqu08:11433/MSSQLSERVER/Adve...,[]
12,[Azure Blob Storage],[],aia-purview-new,azure_blob_service,https://myblobyespe.blob.core.windows.net,[]
13,[Azure Blob Storage],[],aia-purview-new,azure_blob_container,https://myblobyespe.blob.core.windows.net/scan,[]
14,"[Azure Blob Storage, Azure Files, Azure Table ...",[],aia-purview-new,azure_storage_account,https://myblobyespe.core.windows.net,[]
